# Reasoning and Planning Frameworks in Python

This notebook demonstrates four key reasoning and planning frameworks often used in intelligent agent design:

1. **Chain-of-Thought (CoT)** — step-by-step reasoning  
2. **ReAct (Reason + Act + Observe)** — reasoning combined with actions  
3. **Self-Correction and Reflection** — self-review and improvement  
4. **Tree of Thoughts (ToT)** — multi-path reasoning exploration  

Each section includes:
- A short conceptual overview
- A working Python demo (simulated or real)
- Explanations and key takeaways

## 1. Chain-of-Thought (CoT)

**Concept:**  
Chain-of-Thought prompting encourages a model (or reasoning system) to **think step by step** before giving an answer.  
Instead of jumping to conclusions, it explains intermediate reasoning — improving accuracy and transparency.

In [ ]:
"""
=========================================================
CHAIN-OF-THOUGHT (CoT) DEMO
---------------------------------------------------------
This script demonstrates step-by-step reasoning (Chain-of-Thought)
using either a simulated reasoning engine or an LLM API call.
=========================================================
"""

# from openai import OpenAI
# client = OpenAI()

def chain_of_thought_reasoning(question: str, simulate: bool = True) -> str:
    """
    Demonstrates the Chain-of-Thought reasoning pattern.
    
    Args:
        question (str): The problem or question to solve.
        simulate (bool): If True, use a simulated reasoning trace.
                         If False, query a language model.
    
    Returns:
        str: The reasoning trace and final answer.
    """

    if simulate:
        print("Reasoning step by step...\n")

        if "train" in question.lower() and "speed" in question.lower():
            reasoning = [
                "Step 1: Recall that speed = distance / time.",
                "Step 2: The train travels 60 km in 1.5 hours.",
                "Step 3: Compute 60 / 1.5 = 40.",
                "Step 4: Therefore, the average speed is 40 km/h."
            ]
            return "\n".join(reasoning)

        elif "apple" in question.lower() and "total" in question.lower():
            reasoning = [
                "Step 1: John has 3 apples and buys 2 more.",
                "Step 2: Add them together: 3 + 2 = 5.",
                "Step 3: Therefore, John has 5 apples in total."
            ]
            return "\n".join(reasoning)

        else:
            return "No simulation rule defined for this question, but the reasoning would follow a step-by-step logical explanation."

    else:
        response = client.chat.completions.create(
            model="gpt-5",
            messages=[
                {"role": "system", "content": "You are a helpful tutor who always reasons step by step."},
                {"role": "user", "content": f"Question: {question}\nLet's reason step by step before answering."}
            ],
            temperature=0.3
        )
        return response.choices[0].message.content


# Example Run
question = "A train travels 60 km in 1.5 hours. What is its average speed?"
print(chain_of_thought_reasoning(question, simulate=True))

## 2. ReAct (Reason + Act + Observe)

**Concept:**  
ReAct combines reasoning with *actions* and *observations* in a feedback loop.  
This allows an agent to think, perform external actions (like searches), observe the results, and reason again.

In [ ]:
"""
=========================================================
REACT (Reasoning + Acting + Observing) DEMO
---------------------------------------------------------
This script demonstrates a simplified version of the ReAct
framework, where an agent interleaves reasoning steps with
actions and observations.
=========================================================
"""

import time

KNOWLEDGE_BASE = {
    "population of paris": "The population of Paris is approximately 2.1 million (as of 2025).",
    "capital of france": "The capital of France is Paris.",
    "height of eiffel tower": "The Eiffel Tower is about 330 meters tall."
}

def search_knowledge_base(query: str) -> str:
    for key, value in KNOWLEDGE_BASE.items():
        if key in query.lower():
            return value
    return "No results found."

def react_agent(question: str, max_steps: int = 3) -> str:
    print(f"Question: {question}\n")
    observation = ""

    for step in range(1, max_steps + 1):
        print(f"--- Step {step} ---")

        if step == 1:
            thought = f"I need to find information about '{question.lower()}'."
        else:
            thought = f"Based on what I observed, I will check if I now have the answer."
        print(f"Thought: {thought}")

        if "find" in thought.lower():
            action = f"Search knowledge base for: {question}"
            print(f"Action: {action}")
            observation = search_knowledge_base(question)
        else:
            action = "Review observations and summarize."
            print(f"Action: {action}")

        print(f"Observation: {observation}")

        if "No results" not in observation:
            print("Final Answer Found.")
            return f"Answer: {observation}\n"

        print("No conclusive result, reasoning further...\n")
        time.sleep(0.3)

    return "Answer: Unable to find sufficient information."

# Example Run
question = "What is the population of Paris?"
print(react_agent(question))

## 3. Self-Correction and Reflection

**Concept:**  
This framework introduces *self-review*:  
1. The agent produces an initial answer.  
2. It critiques its own answer.  
3. It revises and improves it.

This mirrors how humans reflect and refine their work.

In [ ]:
"""
=========================================================
SELF-CORRECTION AND REFLECTION DEMO
---------------------------------------------------------
This script demonstrates a reasoning pattern where an agent:
1. Generates an initial answer.
2. Reflects on and critiques that answer.
3. Produces a revised, improved version.
=========================================================
"""

def self_correct(question: str, simulate: bool = True) -> str:
    if simulate:
        print(f"Question: {question}\n")
        initial_answer = "Isaac Newton discovered gravity in the 1600s."
        print(f"Initial Answer: {initial_answer}")

        critique = (
            "Critique: Gravity was not discovered by Newton; "
            "he formulated the law of universal gravitation. "
            "Objects always experienced gravity, but Newton mathematically described it."
        )
        print(critique)

        improved_answer = (
            "Corrected Answer: Isaac Newton did not 'discover' gravity; "
            "he formulated the law of universal gravitation in the late 17th century, "
            "explaining how every mass attracts every other mass."
        )

        print(f"\nFinal Answer: {improved_answer}")
        return improved_answer

# Example Run
question = "Who discovered gravity?"
self_correct(question, simulate=True)

## 4. Tree of Thoughts (ToT)

**Concept:**  
Tree of Thoughts generalizes Chain-of-Thought into a *search process*:
- Multiple reasoning paths are explored in parallel.
- Each path is evaluated and scored.
- The best path is chosen as the final reasoning sequence.

This approach is ideal for complex reasoning, puzzles, or planning.

In [ ]:
"""
=========================================================
TREE OF THOUGHTS (ToT) DEMO
---------------------------------------------------------
This script demonstrates how to explore multiple reasoning
paths ("thoughts"), evaluate them, and select the best path.
=========================================================
"""

def generate_branches(thought_path: list, problem: str) -> list:
    last_thought = thought_path[-1] if thought_path else "Start"

    if "raining" in problem.lower():
        return [
            thought_path + ["If it's raining, the ground must be wet. The ground is wet, so it's raining."],
            thought_path + ["The ground is wet, but that doesn't necessarily mean it’s raining (there could be other causes)."],
            thought_path + ["Perhaps it's raining because the ground is wet — but that reasoning is circular."]
        ]
    else:
        return [thought_path + [f"Continue reasoning from: {last_thought}"]]

def evaluate_reasoning_path(thought_path: list) -> float:
    text = " ".join(thought_path).lower()
    if "doesn't necessarily" in text or "not necessarily" in text:
        return 0.9
    elif "so it's raining" in text:
        return 0.4
    else:
        return 0.6

def summarize_best_path(thought_path: list) -> str:
    return " ".join(thought_path)

def tree_of_thoughts(problem: str, depth: int = 2, breadth: int = 3) -> str:
    print(f"Problem: {problem}\n")
    thoughts = [{"path": ["Begin reasoning"], "score": 0.0}]

    for level in range(depth):
        print(f"--- Level {level + 1} ---")
        new_thoughts = []

        for t in thoughts:
            branches = generate_branches(t["path"], problem)
            for b in branches:
                score = evaluate_reasoning_path(b)
                new_thoughts.append({"path": b, "score": score})

        thoughts = sorted(new_thoughts, key=lambda x: x["score"], reverse=True)[:breadth]

        for i, t in enumerate(thoughts, 1):
            print(f"Candidate {i}: Score={t['score']:.2f} | Path={t['path'][-1]}")

    best = thoughts[0]
    print("\nChosen Path:\n" + summarize_best_path(best["path"]))
    return summarize_best_path(best["path"])

# Example Run
problem = "If it's raining, the ground is wet. The ground is wet. Does that mean it's raining?"
tree_of_thoughts(problem)